# Week 3: Semantic Search and Retrieval

## Learning Objectives
- Understand semantic search concepts and applications
- Learn about vector databases and similarity search
- Implement vector search for LLM applications
- Explore different embedding models and techniques
- Introduction to RAG-based architectures

## Table of Contents
1. [Introduction to Semantic Search](#introduction-to-semantic-search)
2. [Vector Embeddings for Search](#vector-embeddings-for-search)
3. [Similarity Metrics](#similarity-metrics)
4. [Vector Databases](#vector-databases)
5. [Implementing Vector Search](#implementing-vector-search)
6. [RAG Architecture](#rag-architecture)
7. [Advanced Retrieval Techniques](#advanced-retrieval-techniques)
8. [Exercises](#exercises)

In [ ]:
# Install required packages
!pip install sentence-transformers faiss-cpu chromadb pinecone-client
!pip install transformers torch numpy pandas matplotlib seaborn
!pip install sklearn datasets openai tiktoken

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import chromadb
import json
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## Introduction to Semantic Search

Semantic search goes beyond keyword matching to understand the meaning and context of queries and documents.

In [ ]:
# Comparison: Traditional vs Semantic Search
def compare_search_approaches():
    print("Traditional Keyword Search vs Semantic Search\n")
    
    comparison = {
        "Aspect": ["Matching", "Understanding", "Synonyms", "Context", "Ranking"],
        "Traditional Search": [
            "Exact keyword matching",
            "No understanding of meaning",
            "Poor synonym handling",
            "No context awareness",
            "Based on keyword frequency"
        ],
        "Semantic Search": [
            "Meaning-based matching",
            "Understands intent and context",
            "Excellent synonym handling",
            "Context-aware",
            "Based on semantic similarity"
        ]
    }
    
    df = pd.DataFrame(comparison)
    print(df.to_string(index=False))
    
    # Example queries
    print("\n\nExample Scenarios:")
    examples = [
        {
            "Query": "machine learning algorithms",
            "Traditional": "Finds documents with exact words 'machine', 'learning', 'algorithms'",
            "Semantic": "Also finds 'AI methods', 'neural networks', 'deep learning models'"
        },
        {
            "Query": "How to cook pasta?",
            "Traditional": "Looks for 'cook' and 'pasta' keywords",
            "Semantic": "Understands cooking intent, finds recipes, preparation methods"
        }
    ]
    
    for i, example in enumerate(examples, 1):
        print(f"\n{i}. Query: '{example['Query']}'")
        print(f"   Traditional: {example['Traditional']}")
        print(f"   Semantic: {example['Semantic']}")

compare_search_approaches()

## Vector Embeddings for Search

Vector embeddings convert text into dense numerical representations that capture semantic meaning.

In [ ]:
# Load a sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample documents
documents = [
    "Machine learning is a subset of artificial intelligence.",
    "Deep learning uses neural networks with multiple layers.",
    "Natural language processing helps computers understand human language.",
    "Computer vision enables machines to interpret visual information.",
    "Reinforcement learning involves training agents through rewards.",
    "Data science combines statistics, programming, and domain expertise.",
    "Python is a popular programming language for AI development.",
    "Transformers revolutionized natural language understanding.",
    "GPT models are large language models for text generation.",
    "BERT is a bidirectional encoder for language understanding."
]

# Generate embeddings
embeddings = model.encode(documents)

print(f"Number of documents: {len(documents)}")
print(f"Embedding dimension: {embeddings.shape[1]}")
print(f"Embeddings shape: {embeddings.shape}")

# Show first document and its embedding
print(f"\nFirst document: '{documents[0]}'")
print(f"First 10 embedding values: {embeddings[0][:10]}")

In [ ]:
# Visualize embeddings using PCA
def visualize_embeddings(embeddings, documents, title="Document Embeddings"):
    # Reduce dimensionality to 2D for visualization
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings)
    
    plt.figure(figsize=(12, 8))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, s=100)
    
    # Add labels for each point
    for i, doc in enumerate(documents):
        # Truncate long documents for readability
        label = doc[:30] + "..." if len(doc) > 30 else doc
        plt.annotate(label, (embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.title(title)
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

visualize_embeddings(embeddings, documents)

## Similarity Metrics

Different metrics to measure similarity between vectors.

In [ ]:
def calculate_similarities(embedding1, embedding2):
    """Calculate different similarity metrics between two embeddings"""
    
    # Cosine similarity
    cosine_sim = cosine_similarity([embedding1], [embedding2])[0][0]
    
    # Dot product
    dot_product = np.dot(embedding1, embedding2)
    
    # Euclidean distance (converted to similarity)
    euclidean_dist = np.linalg.norm(embedding1 - embedding2)
    euclidean_sim = 1 / (1 + euclidean_dist)
    
    # Manhattan distance (converted to similarity)
    manhattan_dist = np.sum(np.abs(embedding1 - embedding2))
    manhattan_sim = 1 / (1 + manhattan_dist)
    
    return {
        'cosine_similarity': cosine_sim,
        'dot_product': dot_product,
        'euclidean_similarity': euclidean_sim,
        'manhattan_similarity': manhattan_sim
    }

# Example: Compare similarities between different document pairs
pairs_to_compare = [
    (0, 1),  # ML and DL (should be similar)
    (0, 6),  # ML and Python (somewhat related)
    (0, 3),  # ML and Computer Vision (related but different)
]

print("Similarity Comparisons:\n")
for idx1, idx2 in pairs_to_compare:
    doc1 = documents[idx1][:50] + "..." if len(documents[idx1]) > 50 else documents[idx1]
    doc2 = documents[idx2][:50] + "..." if len(documents[idx2]) > 50 else documents[idx2]
    
    similarities = calculate_similarities(embeddings[idx1], embeddings[idx2])
    
    print(f"Document 1: {doc1}")
    print(f"Document 2: {doc2}")
    print(f"Similarities:")
    for metric, value in similarities.items():
        print(f"  {metric}: {value:.4f}")
    print("-" * 80)

In [ ]:
# Create a similarity heatmap
def create_similarity_heatmap(embeddings, documents):
    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(embeddings)
    
    # Create short labels for documents
    labels = [doc[:20] + "..." if len(doc) > 20 else doc for doc in documents]
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(similarity_matrix, 
                xticklabels=labels, 
                yticklabels=labels,
                annot=True, 
                fmt='.2f', 
                cmap='YlOrRd',
                square=True)
    plt.title('Document Similarity Matrix (Cosine Similarity)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

create_similarity_heatmap(embeddings, documents)

## Vector Databases

Vector databases are specialized databases optimized for storing and querying high-dimensional vectors.

In [ ]:
# Using FAISS for vector similarity search
class FAISSVectorDB:
    def __init__(self, dimension):
        self.dimension = dimension
        self.index = faiss.IndexFlatIP(dimension)  # Inner product (cosine similarity)
        self.documents = []
        
    def add_documents(self, embeddings, documents):
        """Add documents and their embeddings to the index"""
        # Normalize embeddings for cosine similarity
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings.astype('float32'))
        self.documents.extend(documents)
        
    def search(self, query_embedding, k=5):
        """Search for k most similar documents"""
        # Normalize query embedding
        query_embedding = query_embedding.reshape(1, -1).astype('float32')
        faiss.normalize_L2(query_embedding)
        
        scores, indices = self.index.search(query_embedding, k)
        
        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            if idx < len(self.documents):
                results.append({
                    'rank': i + 1,
                    'document': self.documents[idx],
                    'score': float(score),
                    'index': int(idx)
                })
        return results

# Create FAISS vector database
vector_db = FAISSVectorDB(embeddings.shape[1])
vector_db.add_documents(embeddings.copy(), documents)

print(f"Vector database created with {len(documents)} documents")
print(f"Index size: {vector_db.index.ntotal}")

In [ ]:
# Using ChromaDB as an alternative
def setup_chromadb():
    # Initialize ChromaDB client
    client = chromadb.Client()
    
    # Create a collection
    collection = client.create_collection(
        name="documents",
        metadata={"hnsw:space": "cosine"}  # Use cosine distance
    )
    
    # Add documents to the collection
    collection.add(
        embeddings=embeddings.tolist(),
        documents=documents,
        ids=[f"doc_{i}" for i in range(len(documents))]
    )
    
    return collection

# Set up ChromaDB
try:
    chroma_collection = setup_chromadb()
    print("ChromaDB collection created successfully")
    print(f"Collection count: {chroma_collection.count()}")
except Exception as e:
    print(f"ChromaDB setup failed: {e}")
    chroma_collection = None

## Implementing Vector Search

Let's implement a complete semantic search system.

In [ ]:
class SemanticSearchEngine:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
        self.vector_db = None
        self.documents = []
        
    def index_documents(self, documents):
        """Index a collection of documents"""
        print(f"Indexing {len(documents)} documents...")
        
        # Generate embeddings
        embeddings = self.model.encode(documents, show_progress_bar=True)
        
        # Create vector database
        self.vector_db = FAISSVectorDB(embeddings.shape[1])
        self.vector_db.add_documents(embeddings, documents)
        self.documents = documents
        
        print(f"Indexing complete. {len(documents)} documents indexed.")
        
    def search(self, query, k=5):
        """Search for documents similar to the query"""
        if self.vector_db is None:
            raise ValueError("No documents indexed. Call index_documents() first.")
            
        # Encode query
        query_embedding = self.model.encode([query])[0]
        
        # Search
        results = self.vector_db.search(query_embedding, k)
        
        return results
    
    def explain_search(self, query, k=3):
        """Search and explain the results"""
        results = self.search(query, k)
        
        print(f"Query: '{query}'\n")
        print(f"Top {len(results)} most similar documents:\n")
        
        for result in results:
            print(f"Rank {result['rank']}: (Score: {result['score']:.4f})")
            print(f"Document: {result['document']}")
            print("-" * 60)

# Create search engine and index documents
search_engine = SemanticSearchEngine()
search_engine.index_documents(documents)

In [ ]:
# Test the search engine with different queries
test_queries = [
    "artificial intelligence algorithms",
    "neural network architectures",
    "programming languages for AI",
    "understanding human language",
    "large language models"
]

for query in test_queries:
    print("=" * 80)
    search_engine.explain_search(query, k=3)
    print()

## RAG Architecture

Retrieval-Augmented Generation (RAG) combines retrieval and generation for better responses.

In [ ]:
class SimpleRAGSystem:
    def __init__(self, search_engine):
        self.search_engine = search_engine
        
    def retrieve_and_generate(self, query, k=3):
        """Retrieve relevant documents and generate response"""
        # Retrieve relevant documents
        retrieved_docs = self.search_engine.search(query, k)
        
        # Extract document contents
        context_docs = [doc['document'] for doc in retrieved_docs]
        
        # Create context
        context = "\n".join([f"Document {i+1}: {doc}" for i, doc in enumerate(context_docs)])
        
        # Simple prompt template (in practice, you'd use a real LLM)
        prompt = f"""
Based on the following context documents, answer the question:

Context:
{context}

Question: {query}

Answer: [This would be generated by an LLM like GPT-3/4, Claude, etc.]
"""
        
        return {
            'query': query,
            'retrieved_documents': retrieved_docs,
            'context': context,
            'prompt': prompt
        }
    
    def demonstrate_rag(self, query):
        """Demonstrate the RAG process"""
        result = self.retrieve_and_generate(query)
        
        print(f"Query: {result['query']}\n")
        print("=== RETRIEVAL PHASE ===")
        print("Retrieved Documents:")
        for doc in result['retrieved_documents']:
            print(f"  {doc['rank']}. (Score: {doc['score']:.3f}) {doc['document']}")
        
        print("\n=== GENERATION PHASE ===")
        print("Prompt for LLM:")
        print(result['prompt'])

# Create RAG system
rag_system = SimpleRAGSystem(search_engine)

# Demonstrate RAG
rag_system.demonstrate_rag("What are the main types of machine learning?")

## Advanced Retrieval Techniques

Exploring more sophisticated retrieval methods.

In [ ]:
class HybridSearchEngine:
    """Combines semantic and keyword-based search"""
    
    def __init__(self, semantic_engine, alpha=0.7):
        self.semantic_engine = semantic_engine
        self.alpha = alpha  # Weight for semantic search
        self.documents = semantic_engine.documents
        
    def keyword_search(self, query, documents):
        """Simple keyword-based search using TF-IDF"""
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.metrics.pairwise import cosine_similarity
        
        # Create TF-IDF vectors
        vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
        doc_vectors = vectorizer.fit_transform(documents)
        query_vector = vectorizer.transform([query])
        
        # Calculate similarities
        similarities = cosine_similarity(query_vector, doc_vectors)[0]
        
        # Get top results
        results = []
        for i, score in enumerate(similarities):
            results.append({
                'index': i,
                'document': documents[i],
                'score': float(score)
            })
        
        # Sort by score
        results.sort(key=lambda x: x['score'], reverse=True)
        return results
    
    def hybrid_search(self, query, k=5):
        """Combine semantic and keyword search"""
        # Get semantic search results
        semantic_results = self.semantic_engine.search(query, k=len(self.documents))
        
        # Get keyword search results
        keyword_results = self.keyword_search(query, self.documents)
        
        # Normalize scores
        max_semantic = max([r['score'] for r in semantic_results]) if semantic_results else 1
        max_keyword = max([r['score'] for r in keyword_results]) if keyword_results else 1
        
        # Create combined scores
        combined_scores = {}
        
        # Add semantic scores
        for result in semantic_results:
            idx = result['index']
            norm_score = result['score'] / max_semantic
            combined_scores[idx] = self.alpha * norm_score
        
        # Add keyword scores
        for result in keyword_results:
            idx = result['index']
            norm_score = result['score'] / max_keyword
            if idx in combined_scores:
                combined_scores[idx] += (1 - self.alpha) * norm_score
            else:
                combined_scores[idx] = (1 - self.alpha) * norm_score
        
        # Sort by combined score
        sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
        
        # Format results
        final_results = []
        for rank, (idx, score) in enumerate(sorted_results[:k], 1):
            final_results.append({
                'rank': rank,
                'index': idx,
                'document': self.documents[idx],
                'combined_score': score,
                'semantic_score': next((r['score'] for r in semantic_results if r['index'] == idx), 0),
                'keyword_score': next((r['score'] for r in keyword_results if r['index'] == idx), 0)
            })
        
        return final_results

# Create hybrid search engine
hybrid_engine = HybridSearchEngine(search_engine)

# Test hybrid search
query = "machine learning programming"
results = hybrid_engine.hybrid_search(query, k=5)

print(f"Hybrid Search Results for: '{query}'\n")
for result in results:
    print(f"Rank {result['rank']}: Combined Score: {result['combined_score']:.3f}")
    print(f"  Semantic: {result['semantic_score']:.3f}, Keyword: {result['keyword_score']:.3f}")
    print(f"  Document: {result['document']}")
    print("-" * 60)

In [ ]:
# Query expansion technique
class QueryExpansion:
    def __init__(self, search_engine):
        self.search_engine = search_engine
        
    def expand_query_with_synonyms(self, query):
        """Expand query with synonyms (simplified version)"""
        # Simple synonym dictionary (in practice, use WordNet or word embeddings)
        synonyms = {
            'machine learning': ['artificial intelligence', 'AI', 'ML'],
            'neural networks': ['deep learning', 'neural nets'],
            'programming': ['coding', 'development'],
            'language': ['linguistic', 'text'],
            'computer': ['machine', 'system']
        }
        
        expanded_terms = [query]
        query_lower = query.lower()
        
        for term, syns in synonyms.items():
            if term in query_lower:
                expanded_terms.extend(syns)
        
        return expanded_terms
    
    def search_with_expansion(self, query, k=5):
        """Search using query expansion"""
        expanded_queries = self.expand_query_with_synonyms(query)
        
        print(f"Original query: {query}")
        print(f"Expanded queries: {expanded_queries}")
        
        all_results = {}
        
        # Search with each expanded query
        for exp_query in expanded_queries:
            results = self.search_engine.search(exp_query, k=len(self.search_engine.documents))
            
            for result in results:
                idx = result['index']
                if idx in all_results:
                    all_results[idx]['score'] = max(all_results[idx]['score'], result['score'])
                else:
                    all_results[idx] = result
        
        # Sort and return top k
        sorted_results = sorted(all_results.values(), key=lambda x: x['score'], reverse=True)
        
        return sorted_results[:k]

# Test query expansion
query_expander = QueryExpansion(search_engine)
expanded_results = query_expander.search_with_expansion("machine learning", k=3)

print("\nResults with query expansion:")
for i, result in enumerate(expanded_results, 1):
    print(f"{i}. (Score: {result['score']:.3f}) {result['document']}")

## Exercises

### Exercise 1: Build a Document QA System

In [ ]:
class DocumentQASystem:
    def __init__(self):
        self.search_engine = SemanticSearchEngine()
        self.knowledge_base = []
        
    def add_knowledge_base(self, documents):
        """Add documents to the knowledge base"""
        self.knowledge_base = documents
        self.search_engine.index_documents(documents)
        
    def answer_question(self, question, k=3):
        """Answer a question using the knowledge base"""
        # Retrieve relevant documents
        relevant_docs = self.search_engine.search(question, k)
        
        # Simple answer extraction (in practice, use a more sophisticated method)
        context = " ".join([doc['document'] for doc in relevant_docs])
        
        # For this example, we'll return the most relevant document
        if relevant_docs:
            best_match = relevant_docs[0]
            return {
                'question': question,
                'answer': best_match['document'],
                'confidence': best_match['score'],
                'supporting_docs': relevant_docs
            }
        else:
            return {
                'question': question,
                'answer': "I don't have enough information to answer this question.",
                'confidence': 0.0,
                'supporting_docs': []
            }

# Create QA system with AI knowledge base
ai_knowledge = [
    "Machine learning is a subset of artificial intelligence that enables computers to learn without being explicitly programmed.",
    "Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data.",
    "Deep learning is a subset of machine learning that uses neural networks with multiple layers.",
    "Natural language processing enables computers to understand, interpret, and generate human language.",
    "Computer vision allows machines to interpret and understand visual information from the world.",
    "Reinforcement learning involves training agents to make decisions through trial and error.",
    "Neural networks are inspired by the human brain and consist of interconnected nodes called neurons.",
    "Transformers are a type of neural network architecture that has revolutionized NLP.",
    "Large language models like GPT are trained on vast amounts of text data.",
    "AI ethics is important to ensure responsible development and deployment of AI systems."
]

qa_system = DocumentQASystem()
qa_system.add_knowledge_base(ai_knowledge)

# Test the QA system
questions = [
    "What is machine learning?",
    "How does deep learning work?",
    "What are transformers in AI?",
    "Why is AI ethics important?"
]

for question in questions:
    result = qa_system.answer_question(question)
    print(f"Q: {result['question']}")
    print(f"A: {result['answer']}")
    print(f"Confidence: {result['confidence']:.3f}")
    print("-" * 60)

### Exercise 2: Evaluate Search Quality

In [ ]:
def evaluate_search_quality(search_engine, test_cases):
    """Evaluate search quality using test cases"""
    
    total_precision = 0
    total_recall = 0
    total_mrr = 0  # Mean Reciprocal Rank
    
    for case in test_cases:
        query = case['query']
        relevant_docs = set(case['relevant_doc_indices'])
        
        # Get search results
        results = search_engine.search(query, k=5)
        retrieved_indices = set([r['index'] for r in results])
        
        # Calculate precision and recall
        if retrieved_indices:
            precision = len(relevant_docs & retrieved_indices) / len(retrieved_indices)
            recall = len(relevant_docs & retrieved_indices) / len(relevant_docs) if relevant_docs else 0
        else:
            precision = recall = 0
        
        # Calculate MRR
        mrr = 0
        for rank, result in enumerate(results, 1):
            if result['index'] in relevant_docs:
                mrr = 1 / rank
                break
        
        total_precision += precision
        total_recall += recall
        total_mrr += mrr
        
        print(f"Query: {query}")
        print(f"  Precision: {precision:.3f}, Recall: {recall:.3f}, MRR: {mrr:.3f}")
        print(f"  Retrieved: {list(retrieved_indices)}, Relevant: {list(relevant_docs)}")
    
    # Calculate averages
    n_cases = len(test_cases)
    avg_precision = total_precision / n_cases
    avg_recall = total_recall / n_cases
    avg_mrr = total_mrr / n_cases
    f1_score = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
    
    print("\n=== OVERALL PERFORMANCE ===")
    print(f"Average Precision: {avg_precision:.3f}")
    print(f"Average Recall: {avg_recall:.3f}")
    print(f"F1 Score: {f1_score:.3f}")
    print(f"Mean Reciprocal Rank: {avg_mrr:.3f}")

# Create test cases
test_cases = [
    {
        'query': 'neural networks and deep learning',
        'relevant_doc_indices': [1, 7]  # Deep learning and Transformers
    },
    {
        'query': 'natural language understanding',
        'relevant_doc_indices': [2, 7, 9]  # NLP, Transformers, BERT
    },
    {
        'query': 'programming for AI',
        'relevant_doc_indices': [6]  # Python programming
    }
]

# Evaluate search engine
evaluate_search_quality(search_engine, test_cases)

## Summary

In this module, we covered:
- Semantic search concepts and applications
- Vector embeddings and similarity metrics
- Vector databases (FAISS, ChromaDB)
- Implementation of semantic search systems
- RAG (Retrieval-Augmented Generation) architecture
- Advanced retrieval techniques (hybrid search, query expansion)
- Search quality evaluation metrics

## Next Steps
In the next module, we'll build a complete search engine from scratch, implementing more sophisticated indexing and ranking algorithms.

## Additional Resources
- [Sentence Transformers Documentation](https://www.sbert.net/)
- [FAISS Documentation](https://faiss.ai/)
- [ChromaDB Documentation](https://docs.trychroma.com/)
- [Pinecone Vector Database](https://www.pinecone.io/)
- [RAG Paper: Retrieval-Augmented Generation](https://arxiv.org/abs/2005.11401)
- [Dense Passage Retrieval](https://arxiv.org/abs/2004.04906)
- [ColBERT: Efficient and Effective Passage Search](https://arxiv.org/abs/2004.12832)